
# A1111 Extension Manager (UI)

Install or update **Automatic1111** extensions into the WebUI **code tree** so they load properly:

- **Extensions code dir**: `/opt/stable-diffusion-webui/extensions`
- **Data/models root**: `/workspace/a1111` (models for ControlNet / AnimateDiff / InsightFace, etc.)

### How it works
- **Defaults**: always installed when you click a button (safe, lightweight helpers).
- **Recommended**: you choose via checkboxes, then click **Install Selected**.
- **Extra URLs**: paste any Git repo URLs (one per line) to install as well.

> After installing or updating, open the WebUI and go to **Settings → Reload UI** (or restart A1111) to load new extensions.


In [ ]:

import ipywidgets as W
from IPython.display import display, clear_output
import subprocess
from urllib.parse import urlparse
from pathlib import Path

# ---- Paths ----
CODE_EXT_DIR = Path("/opt/stable-diffusion-webui/extensions")   # WebUI code tree (must be here)
DATA_ROOT    = Path("/workspace/a1111")                          # Where heavy models can live

CODE_EXT_DIR.mkdir(parents=True, exist_ok=True)
(DATA_ROOT / "models").mkdir(parents=True, exist_ok=True)

# ---- Extension lists ----

DEFAULTS = [
    {"label": "Aspect Ratio Helper", "url": "https://github.com/alemelis/sd-webui-ar"},
    {"label": "Tag Autocomplete",    "url": "https://github.com/DominikDoom/a1111-sd-webui-tagcomplete"},
    {"label": "Images Browser",      "url": "https://github.com/AlUlkesh/stable-diffusion-webui-images-browser"},
    {"label": "Config Presets",      "url": "https://github.com/Zyin055/Config-Presets"},
]

RECOMMENDED = [
    {"label": "ADetailer",            "url": "https://github.com/Bing-su/adetailer",                            "enable": True},
    {"label": "Ultimate SD Upscale",  "url": "https://github.com/Coyote-A/ultimate-upscale-for-automatic1111", "enable": True},
    {"label": "ControlNet",           "url": "https://github.com/Mikubill/sd-webui-controlnet",                 "enable": False},
    {"label": "Dynamic Prompts",      "url": "https://github.com/adieyal/sd-dynamic-prompts",                   "enable": False},
    {"label": "OpenPose Editor",      "url": "https://github.com/fkunn1326/openpose-editor",                    "enable": False},
]

def sh(cmd, cwd=None):
    print("$", " ".join(cmd))
    return subprocess.run(cmd, cwd=cwd, check=True)

def repo_basename(url: str) -> str:
    path = urlparse(url).path.rstrip("/")
    name = path.split("/")[-1]
    return name[:-4] if name.endswith(".git") else name

def install_or_update(url: str, base: Path) -> Path:
    name = repo_basename(url)
    dst = base / name
    if not dst.exists():
        print(f"\n[install] {name} from {url}")
        sh(["git", "clone", "--depth", "1", url, str(dst)])
    else:
        print(f"\n[update] {name}")
        try:
            sh(["git", "-C", str(dst), "reset", "--hard"])
            sh(["git", "-C", str(dst), "pull", "--ff-only"])
        except subprocess.CalledProcessError:
            print("  (non-ff; attempting rebase)")
            sh(["git", "-C", str(dst), "pull", "--rebase", "--autostash"])
    return dst

# ---- UI widgets ----
hdr = W.HTML("<h3>Extensions</h3>")
tbl_hdr = W.HBox([
    W.HTML("<b>Name</b>", layout=W.Layout(width="40%")),
    W.HTML("<b>Status</b>", layout=W.Layout(width="20%")),
    W.HTML("<b>Homepage</b>", layout=W.Layout(width="40%")),
])

# Checkboxes for recommended
checks = []
rows = []
for ext in RECOMMENDED:
    cb = W.Checkbox(description=ext["label"], value=bool(ext.get("enable", False)))
    checks.append((cb, ext))
    rows.append(W.HBox([cb, W.HTML("not installed"), W.HTML(f"<a href='{ext['url']}' target='_blank'>GitHub</a>")]))

# Defaults list (shown for transparency)
defaults_list = W.HTML(
    "<ul style='margin-top:0'>" + "".join([
        f"<li><a href='{d['url']}' target='_blank'>{d['label']}</a></li>"
        for d in DEFAULTS
    ]) + "</ul>"
)

extra_label = W.HTML("<b>Extra Git repo URLs (one per line):</b>")
extra_text  = W.Textarea(placeholder="https://github.com/user/repo-one\nhttps://github.com/user/repo-two",
                         layout=W.Layout(width="100%", height="100px"))

btn_install_defaults = W.Button(description="Install Defaults")
btn_install_selected = W.Button(description="Install Selected")
btn_install_all      = W.Button(description="Install All (Defaults + Selected + Extra)", button_style="primary")
out = W.Output(layout=W.Layout(border="1px solid #444", padding="6px", max_height="350px", overflow_y="auto"))

def _do_install(do_defaults=False, do_selected=False, do_extra=False):
    with out:
        clear_output()
        print("Extensions dir (code):", CODE_EXT_DIR)
        print("Data root (models):   ", DATA_ROOT)
        errors = []
        installed = []

        if do_defaults:
            print("\n==> Installing defaults ...")
            for d in DEFAULTS:
                try:
                    installed.append(install_or_update(d["url"], CODE_EXT_DIR))
                except Exception as e:
                    errors.append((d["label"], str(e)))

        if do_selected:
            print("\n==> Installing selected ...")
            for cb, ext in checks:
                if cb.value:
                    try:
                        installed.append(install_or_update(ext["url"], CODE_EXT_DIR))
                    except Exception as e:
                        errors.append((ext["label"], str(e)))

        if do_extra:
            print("\n==> Installing extra URLs ...")
            for line in extra_text.value.splitlines():
                u = line.strip()
                if u:
                    try:
                        installed.append(install_or_update(u, CODE_EXT_DIR))
                    except Exception as e:
                        errors.append((u, str(e)))

        print("\n=== Summary ===")
        for p in installed:
            print("✔", p)
        if errors:
            print("\nErrors:")
            for name, msg in errors:
                print("✖", name, "→", msg)
        print("\nDone. In WebUI: Settings → Reload UI (or restart A1111).")

btn_install_defaults.on_click(lambda b: _do_install(do_defaults=True, do_selected=False, do_extra=False))
btn_install_selected.on_click(lambda b: _do_install(do_defaults=False, do_selected=True, do_extra=False))
btn_install_all.on_click(lambda b: _do_install(do_defaults=True, do_selected=True, do_extra=True))

ui = W.VBox([
    hdr,
    W.HTML("<b>Defaults</b> (always installed when you click a button):"),
    defaults_list,
    W.HTML("<b>Recommended</b> (select any):"),
    tbl_hdr,
    W.VBox(rows),
    W.HTML("<hr>"),
    extra_label,
    extra_text,
    W.HBox([btn_install_defaults, btn_install_selected, btn_install_all], layout=W.Layout(justify_content="flex-start", gap="8px", margin="8px 0")),
    out
])

display(ui)
